#### testing out the encoder replacement in this section and see if it works or not

##### testing by adding the horizontal and vertical sections together 

##### the inference speed and model size decrease

In [1]:
import os 
import cv2 
import argparse
import torch
import torchvision.transforms as T

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from repnet import utils, plots
from repnet.model import RepNet

import event_tools.tools as tl 
import visualization as vz
import event_tools.e2v as e2v


In [2]:
event_class = "class5"
# event_user = "user02_fluorescent"
event_user = "user02_fluorescent_led"
# event_user = "user02_lab"
# event_user = "user02_led"
# event_user = "user02_natural"

event_csv_file_name = f"./event_csv/split_data/{event_class}/{event_user}.csv"

count_data = {
    "class": event_class,
    "condition": event_user,
    "count": 0,
}

# output count info 
out_csv_file = "encoder_replaced_output.csv"

if os.path.exists(out_csv_file):
    try: 
        out_df = pd.read_csv(out_csv_file)
    except: 
        out_df = pd.DataFrame([count_data])
else: 
    with open(out_csv_file, "w") as f:
        f.write("class,condition,count\n")
    out_df = pd.read_csv(out_csv_file)


In [3]:
# out_df = pd.concat([out_df, pd.DataFrame(columns=["class", "condition", "count"])])

out_df

,class,condition,count
0,class2,user02_fluorescent_led,10
1,class2,user02_fluorescent,14
2,class2,user02_lab,9
3,class2,user02_led,9
4,class2,user02_natural,16
5,class3,user02_fluorescent_led,12
6,class3,user02_fluorescent,11
7,class3,user02_lab,10
8,class3,user02_led,12
9,class3,user02_natural,11


In [4]:
# video_path = './'
events = tl.load_event_data(
    # './event_csv/split_data/class7/user02_lab.csv')  # Replace with your actual file path
    event_csv_file_name) # Replace with your actual file path

In [5]:
# repnet model variables 
weights = './pytorch_weights.pth'
device = 'cuda'
strides = [1, 2, 3, 4 , 8]


OUT_VISUALIZATIONS_DIR = './visualization/'


In [6]:
# spiking variables

width=128
height=128

horizontal_member_potential = np.zeros(width) 
vertical_member_potential = np.zeros(width) 

horizontal_neurons = np.zeros(width).reshape((1, -1))
vertical_neurons = np.zeros(height).reshape((1, -1))

# horizontal_member_potential = torch.zeros((6, width), device=device) 
# vertical_member_potential = torch.zeros((64, width), device=device) 

# horizontal_neurons = torch.zeros((64, width), device=device).reshape((1, -1))
# vertical_neurons = torch.zeros((64, height), device=device).reshape((1, -1))



decay = .009 # 0.009
batch_size = 1 #  64 #  
random_wt_2d = lambda _row, _col: torch.ones((_row, _col), device=device) * 0.5 # np.round(np.random.rand(_row, _col), 3)
weight_input_horizontal_hidden = random_wt_2d(batch_size, height) # np.ones((1, height)) * weight_scale_1
weight_input_horizontal_hidden_1 = random_wt_2d(height, height//2) # np.ones((height, height//2)) * weight_scale_1

# not considering the height and width for the image as they are symmetrical 
weight_input_vertical_hidden = random_wt_2d(batch_size, height) # np.ones((1, height)) * weight_scale_1
weight_input_vertical_hidden_1 = random_wt_2d(height, height//2) # np.ones((height, height//2)) * weight_scale_1



peak = 255 
spike = 1 # 255 
threshold = 0.8 

fps = 60
frame_per_second  = 1/fps

In [7]:
# e2v.save_event_as_video(events,name="event.mp4", frame_per_second=frame_per_second, decay=0.009)

In [8]:
# read frames and apply preprocessing 
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((112, 112)),
    T.ToTensor(),
    T.Normalize(mean=0.5, std=0.5),      
])

raw_frames, frames = [], [] 

for idx, frame_image in enumerate(
    e2v.get_event_frame(events, frame_per_second = frame_per_second, 
        decay= decay)):
    cv2.imshow('raw' , frame_image)
    frame_image = frame_image #.T
    frames.append(torch.from_numpy(frame_image).float().to(device))
    
    frame_image = np.clip(frame_image * 255, 0, 255).astype(np.uint8)
    frame_image = cv2.cvtColor(frame_image, cv2.COLOR_GRAY2BGR)
    cv2.imshow('test', cv2.resize(frame_image, [frame_image.shape[0] * 10, frame_image.shape[0] * 10] )) 
    
    if(cv2.waitKey(1) == ord('q')):
        break 
    raw_frames.append(frame_image)
    # frame = transform(frame_image)
    # frames.append(frame)
cv2.destroyAllWindows()

In [9]:
len(raw_frames), len(frames), type(frames)

(326, 326, list)

In [10]:
# Load model
model = RepNet()
state_dict = torch.load(weights)
model.load_state_dict(state_dict)

if hasattr(model, 'encoder'):
    del model.encoder

model.eval()
model.to(device)

Using cache found in /home/rlwagun/.cache/torch/hub/huggingface_pytorch-image-models_main


RepNet(
  (temporal_conv): Sequential(
    (0): Conv3d(1024, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(3, 1, 1), dilation=(3, 1, 1))
    (1): BatchNorm3d(512, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AdaptiveMaxPool3d(output_size=(None, 1, 1))
    (4): Flatten(start_dim=2, end_dim=4)
  )
  (tsm_conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (period_length_head): Sequential(
    (0): TranformerLayer(
      (input_projection): Linear(in_features=2048, out_features=512, bias=True)
      (transformer_layer): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=51

In [11]:
import torch
import psutil

# GPU Memory Usage
gpu_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MB
print(f"GPU Memory Usage: {gpu_memory:.2f} MB")

# CPU RAM Usage
ram_usage = psutil.Process().memory_info().rss / (1024 * 1024)  # Convert to MB
print(f"CPU RAM Usage: {ram_usage:.2f} MB")

def get_model_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())  # Model parameters
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())  # Model buffers (like BatchNorm stats)
    total_size_mb = (param_size + buffer_size) / (1024 * 1024)  # Convert to MB
    return total_size_mb

model_size = get_model_size(model)
print(f"Model Size: {model_size:.2f} MB")

GPU Memory Usage: 98.57 MB
CPU RAM Usage: 850.85 MB
Model Size: 78.13 MB


In [12]:
# frames = torch.stack(frames)
len(frames)

326

In [13]:
import subprocess
import re

def get_jetson_power_snapshot():
    """Returns Jetson average power usage (Watts) from tegrastats."""
    try:
        result = subprocess.run(['tegrastats'], stdout=subprocess.PIPE, stderr=subprocess.DEVNULL, timeout=1)
        output = result.stdout.decode()

        # Look for VDD_IN or other power fields
        match = re.search(r'VDD_IN (\d+)mW', output)
        if match:
            return int(match.group(1)) / 1000  # Convert mW to W

        print("Could not find VDD_IN in tegrastats output.")
        return 0.0
    except Exception as e:
        print("Error reading Jetson power:", e)
        return 0.0
# Example usage
print("Power draw (W):", get_jetson_power_snapshot())


Error reading Jetson power: Command '['tegrastats']' timed out after 1 seconds
Power draw (W): 0.0


In [14]:
%%timeit

import torch

# Initialize potentials
horizontal_member_potential = torch.zeros((1, width), device=device)
vertical_member_potential = torch.zeros((1, width), device=device)

best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = None, None, None, None, None, None

spikes_collection = []
# Process frames sequentially while optimizing computations
# for idx in range(frames.shape[0]):  # Stride-based loop
for idx in range(len(frames)):  # Stride-based loop
    frame_image = frames[idx]

    # Update potentials sequentially
    horizontal_member_potential.mul_(decay).add_(weight_input_horizontal_hidden @ frame_image)
    vertical_member_potential.mul_(decay).add_(weight_input_vertical_hidden @ frame_image.T)

    # Generate spikes
    spike_mask_h = horizontal_member_potential > (spike * threshold)
    spike_mask_v = vertical_member_potential > (spike * threshold)

    horizontal_neurons = spike_mask_h.float()
    vertical_neurons = spike_mask_v.float()

    horizontal_member_potential.masked_fill_(spike_mask_h, 0).relu_()
    vertical_member_potential.masked_fill_(spike_mask_v, 0).relu_()

    # Collect spikes
    spikes_collection.append(torch.cat([horizontal_neurons, vertical_neurons], dim=-1))


for stride in strides:
    # apply stride 
    stride_frames = spikes_collection[::stride]
    stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
    if len(stride_frames) < 64:
        continue # Skip this stride if there are not enough frames 
        
    raw_period_length, raw_periodicity_score, embeddings = [], [], []

    # # Process batch inference every 64 frames
    # if len(stride_frames) == 64:
    with torch.no_grad():
        spikes_0 = torch.stack(stride_frames).view(-1, 64, 128 * 2).to(device)
        batch_period_length, batch_periodicity, batch_embeddings = model(spikes_0)

        # Store results
        raw_period_length.append(batch_period_length[0].cpu())
        raw_periodicity_score.append(batch_periodicity[0].cpu())
        embeddings.append(batch_embeddings[0].cpu())

    # Post-process results
    if raw_period_length and raw_periodicity_score:
        raw_period_length, raw_periodicity_score, embeddings = map(torch.cat, [raw_period_length, raw_periodicity_score, embeddings])
        confidence, period_length, period_count, periodicity_score = model.get_counts(raw_period_length, raw_periodicity_score, stride)

        if best_confidence is None or confidence > best_confidence:
            best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = stride, confidence, period_length, period_count, periodicity_score, embeddings

# Final check
if best_stride is None:
    raise RuntimeError('Stride values too large; no 64-frame chunk could be sampled. Try different --strides values.')

print(f'Predicted period length: {best_period_length/fps:.1f} sec (~{int(best_period_length)} frames) with confidence {best_confidence:.2f}, using a stride of {best_stride}.')


Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
352 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
def run():
    # Initialize potentials
    horizontal_member_potential = torch.zeros((1, width), device=device)
    vertical_member_potential = torch.zeros((1, width), device=device)

    best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = None, None, None, None, None, None

    spikes_collection = []
    # Process frames sequentially while optimizing computations
    # for idx in range(frames.shape[0]):  # Stride-based loop
    for idx in range(len(frames)):  # Stride-based loop
        frame_image = frames[idx]

        # Update potentials sequentially
        horizontal_member_potential.mul_(decay).add_(weight_input_horizontal_hidden @ frame_image)
        vertical_member_potential.mul_(decay).add_(weight_input_vertical_hidden @ frame_image.T)

        # Generate spikes
        spike_mask_h = horizontal_member_potential > (spike * threshold)
        spike_mask_v = vertical_member_potential > (spike * threshold)

        horizontal_neurons = spike_mask_h.float()
        vertical_neurons = spike_mask_v.float()

        horizontal_member_potential.masked_fill_(spike_mask_h, 0).relu_()
        vertical_member_potential.masked_fill_(spike_mask_v, 0).relu_()

        # Collect spikes
        spikes_collection.append(torch.cat([horizontal_neurons, vertical_neurons], dim=-1))


    for stride in strides:
        # apply stride 
        stride_frames = spikes_collection[::stride]
        stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
        if len(stride_frames) < 64:
            continue # Skip this stride if there are not enough frames 
            
        raw_period_length, raw_periodicity_score, embeddings = [], [], []

        # # Process batch inference every 64 frames
        # if len(stride_frames) == 64:
        with torch.no_grad():
            spikes_0 = torch.stack(stride_frames).view(-1, 64, 128 * 2).to(device)
            batch_period_length, batch_periodicity, batch_embeddings = model(spikes_0)

            # Store results
            raw_period_length.append(batch_period_length[0].cpu())
            raw_periodicity_score.append(batch_periodicity[0].cpu())
            embeddings.append(batch_embeddings[0].cpu())

        # Post-process results
        if raw_period_length and raw_periodicity_score:
            raw_period_length, raw_periodicity_score, embeddings = map(torch.cat, [raw_period_length, raw_periodicity_score, embeddings])
            confidence, period_length, period_count, periodicity_score = model.get_counts(raw_period_length, raw_periodicity_score, stride)

            if best_confidence is None or confidence > best_confidence:
                best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = stride, confidence, period_length, period_count, periodicity_score, embeddings

    # Final check
    if best_stride is None:
        raise RuntimeError('Stride values too large; no 64-frame chunk could be sampled. Try different --strides values.')

    print(f'Predicted period length: {best_period_length/fps:.1f} sec (~{int(best_period_length)} frames) with confidence {best_confidence:.2f}, using a stride of {best_stride}.')


In [23]:
import time
from jtop import jtop

num_samples=20
interval=0.05
power_samples = []
inference_times = []

with jtop() as jetson:
    if not jetson.ok():
        print("Jetson is not ready.")
        # return {}

    # Wait for sensors to stabilize
    time.sleep(1)

    for i in range(num_samples):
        if not jetson.ok():
            break

        # Run inference and measure time
        t0 = time.time()
        result = run()
        t1 = time.time()

        inference_time_ms = (t1 - t0) * 1000  # convert to milliseconds
        inference_times.append(inference_time_ms)

        try:
            power = jetson.power['tot']['power'] / 1000.0  # mW to W
            power_samples.append(power)
        except Exception:
            continue

        time.sleep(interval)

avg_inference_time_ms = sum(inference_times) / len(inference_times)
avg_power = sum(power_samples) / len(power_samples) if power_samples else 0

print(f"Avg Inference Time: {avg_inference_time_ms:.2f} ms")
print(f"Avg Power: {avg_power:.2f} W")



Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period length: 0.8 sec (~45 frames) with confidence 0.98, using a stride of 2.
Predicted period leng

In [ ]:
import time
import subprocess
import re

def get_jetson_power_snapshot():
    """Returns Jetson average power usage (Watts) from tegrastats."""
    try:
        result = subprocess.run(['tegrastats'], stdout=subprocess.PIPE, stderr=subprocess.DEVNULL) #, timeout=1)
        output = result.stdout.decode()

        # Look for VDD_IN or other power fields
        match = re.search(r'VDD_IN (\d+)mW', output)
        if match:
            return int(match.group(1)) / 1000  # Convert mW to W

        print("Could not find VDD_IN in tegrastats output.")
        return 0.0
    except Exception as e:
        print("Error reading Jetson power:", e)
        return 0.0
    
def run_inference_with_metrics(run_fn, num_trials=5):
    """Runs inference and logs average time, power, and estimated energy."""
    total_time = 0.0
    total_power = 0.0

    for i in range(num_trials):
        print(f"\n[Trial {i+1}]")

        # Get pre-inference power
        power_before = get_jetson_power_snapshot()

        # Time the inference
        start = time.time()
        run_fn()  # Your actual inference method
        end = time.time()

        # Get post-inference power
        power_after = get_jetson_power_snapshot()

        elapsed = end - start
        avg_power = (power_before + power_after) / 2
        energy = avg_power * elapsed

        print(f"Inference Time: {elapsed:.4f} sec")
        print(f"Avg Power: {avg_power:.4f} W")
        print(f"Estimated Energy: {energy:.4f} J")

        total_time += elapsed
        total_power += avg_power

    print("\n=== Overall Averages ===")
    avg_time = total_time / num_trials
    avg_power = total_power / num_trials
    avg_energy = avg_time * avg_power
    print(f"Avg Inference Time: {avg_time:.4f} sec")
    print(f"Avg Power: {avg_power:.4f} W")
    print(f"Avg Energy: {avg_energy:.4f} J")

# # Replace this with your actual inference method
# def run():
#     # Example workload (simulate inference)
#     import torch
#     x = torch.randn(1, 3, 224, 224).cuda()
#     model = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1).cuda()
#     with torch.no_grad():
#         for _ in range(100):
#             y = model(x)

# Run the test
run_inference_with_metrics(run, num_trials=5)



[Trial 1]


KeyboardInterrupt: 

In [ ]:
def count_parameters(model):
    # Total number of parameters in the model
    return sum(p.numel() for p in model.parameters())

def get_model_size(model):
    """Return model size in MB."""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    return param_size / (1024 ** 2)

def get_gpu_memory():
    """Return GPU memory usage in MB."""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / (1024 ** 2)
    return 0.0

def get_cpu_memory():
    """Return RAM usage in MB."""
    return psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

print(f"model size {get_model_size(model)}, gpu memory {get_gpu_memory()}, cpu mem {get_cpu_memory()}")
print(f"SNN-based RepNet parameters: {count_parameters(model) / 1e6:.2f}M")

model size 78.12292861938477, gpu memory 106.69287109375, cpu mem 1043.9609375
SNN-based RepNet parameters: 20.48M


In [26]:
# Generate video with counts
rep_frames = plots.plot_repetitions(raw_frames[:len(best_period_count)], best_period_count.tolist(), best_periodicity_score.tolist() if not True else None)
video = cv2.VideoWriter(os.path.join(OUT_VISUALIZATIONS_DIR, 'repetitions.mp4'), cv2.VideoWriter_fourcc(*'mp4v'), fps, rep_frames[0].shape[:2][::-1])
for frame in rep_frames:
    video.write(frame)
video.release()

print('Done')

NameError: name 'best_period_count' is not defined

In [61]:
len(best_period_count)

256

In [62]:
len(raw_frames), len(best_period_count)

(312, 256)

In [65]:
# best_period_count.tolist()

In [64]:
best_period_count.tolist()[-1]
round(best_period_count.tolist()[-1])

11

In [ ]:
count_data["count"] = round(best_period_count.tolist()[-1])
try:
    if (out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count']):
        out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count'] = count_data["count"]
except:
    out_df = pd.concat([out_df, pd.DataFrame([count_data])])

In [ ]:
out_df.to_csv(out_csv_file, index=False)

In [31]:
out_df
# out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count'] = count_data["count"]

,class,condition,count
0,class2,user02_fluorescent_led,10
1,class2,user02_fluorescent,14
2,class2,user02_lab,9
3,class2,user02_led,9
4,class2,user02_natural,16
5,class3,user02_fluorescent_led,12
6,class3,user02_fluorescent,11
7,class3,user02_lab,10
8,class3,user02_led,12
9,class3,user02_natural,11


In [12]:
import torch
import psutil

# GPU Memory Usage
gpu_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MB
print(f"GPU Memory Usage: {gpu_memory:.2f} MB")

# CPU RAM Usage
ram_usage = psutil.Process().memory_info().rss / (1024 * 1024)  # Convert to MB
print(f"CPU RAM Usage: {ram_usage:.2f} MB")


GPU Memory Usage: 106.76 MB
CPU RAM Usage: 1406.93 MB
